# Import Lib

In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from keras.models import Model
from keras.layers import *
from six.moves import range

Using TensorFlow backend.


# Parameters Config

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [9]:
TRAINING_SIZE = 80000
DIGITS = 3
DIGITS_LABEL = 6
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789* '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)

In [6]:
ctable.indices_char

{0: ' ',
 1: '*',
 2: '0',
 3: '1',
 4: '2',
 5: '3',
 6: '4',
 7: '5',
 8: '6',
 9: '7',
 10: '8',
 11: '9'}

# Data Generation

In [7]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}*{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a * b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 80000


In [8]:
print(questions[:5], expected[:5])

['0*22   ', '671*179', '3*211  ', '8*734  ', '78*0   '] ['0   ', '120109', '633 ', '5872', '0   ']


# Processing

In [10]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS_LABEL, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS_LABEL)

Vectorization...


In [11]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:20000]
train_y = y[:20000]
test_x = x[20000:]
test_y = y[20000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(18000, 7, 12)
(18000, 6, 12)
Validation Data:
(2000, 7, 12)
(2000, 6, 12)
Testing Data:
(60000, 7, 12)
(60000, 6, 12)


In [12]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False False False False False False False
    True]
  [False False False False False False False False False False  True
   False]
  [False False False  True False False False False False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False False  True False False False
   False]
  [False False False False False False False False False  True False
   False]
  [ True False False False False False False False False False False
   False]]

 [[False False False False False False False False False False False
    True]
  [False False False False False False False False False False False
    True]
  [False  True False False False False False False False False False
   False]
  [False False False False False  True False False False False False
   False]
  [False False False False False False  True False False False False
   False]
  [False False False False False False Fal

# Build Model

In [13]:
x_train.shape[1:]

(7, 12)

In [14]:
print('Build model...')

##### Build your own model here ############
# input to LSTM should have shape (nb_of_samples, seq_len, features)
# 
def my_model(input_data):
    
    model = Sequential()
    model.add(layers.LSTM(HIDDEN_SIZE, input_shape=(input_data[1], input_data[2]), name="lstm_1") )
    model.add(layers.RepeatVector(DIGITS_LABEL, name="repeat_vector"))
    model.add(layers.LSTM(HIDDEN_SIZE, return_sequences=True, name="lstm_2"))
    model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax'), name="time_distributed"))

    model.summary()
    
    return model

Build model...


In [15]:
x_train.shape[1]

7

In [16]:
model = my_model(x_train.shape)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 6, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 6, 128)            131584    
_________________________________________________________________
time_distributed (TimeDistri (None, 6, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


# Training

In [17]:
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 263us/step - loss: 1.8747 - acc: 0.1505 - val_loss: 1.8046 - val_acc: 0.1750
Q 494*68  T 33592  ☒ 134444
Q 400*459 T 183600 ☒ 100000
Q 70*890  T 62300  ☒ 100000
Q 423*476 T 201348 ☒ 133444
Q 907*922 T 836254 ☒ 134400
Q 35*341  T 11935  ☒ 133500
Q 56*90   T 5040   ☒ 100000
Q 4*294   T 1176   ☒ 144444
Q 921*874 T 804954 ☒ 134444
Q 425*139 T 59075  ☒ 133355

--------------------------------------------------
Iteration 1
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 192us/step - loss: 1.7844 - acc: 0.1919 - val_loss: 1.7859 - val_acc: 0.1675
Q 21*197  T 4137   ☒ 113366
Q 924*763 T 705012 ☒ 133366
Q 282*79  T 22278  ☒ 133666
Q 288*9   T 2592   ☒ 186666
Q 670*7   T 4690   ☒ 100000
Q 27*305  T 8235   ☒ 100000
Q 80*943  T 75440  ☒ 130000
Q 32*329  T 10528  ☒ 113

18000/18000 [==============================] - 4s 206us/step - loss: 1.3287 - acc: 0.3729 - val_loss: 1.3341 - val_acc: 0.3747
Q 375*27  T 10125  ☒ 107550
Q 69*717  T 49473  ☒ 587119
Q 497*790 T 392630 ☒ 374410
Q 52*261  T 13572  ☒ 156442
Q 3*645   T 1935   ☒ 11250 
Q 150*589 T 88350  ☒ 107500
Q 531*913 T 484803 ☒ 547667
Q 59*673  T 39707  ☒ 474139
Q 172*2   T 344    ☒ 482   
Q 416*4   T 1664   ☒ 1728  

--------------------------------------------------
Iteration 14
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 199us/step - loss: 1.3101 - acc: 0.3817 - val_loss: 1.3187 - val_acc: 0.3747
Q 969*86  T 83334  ☒ 710040
Q 937*24  T 22488  ☒ 296220
Q 366*496 T 181536 ☒ 191552
Q 38*543  T 20634  ☒ 214222
Q 211*824 T 173864 ☒ 166542
Q 493*9   T 4437   ☒ 49033 
Q 373*363 T 135399 ☒ 131001
Q 29*177  T 5133   ☒ 5493  
Q 33*217  T 7161   ☒ 7097  
Q 232*98  T 22736  ☒ 296522

--------------------------------------------------
Iteration 

Q 458*99  T 45342  ☒ 444320
Q 852*945 T 805140 ☒ 776550
Q 645*66  T 42570  ☒ 433200
Q 93*747  T 69471  ☒ 72207 
Q 84*848  T 71232  ☒ 77712 
Q 68*739  T 50252  ☒ 52228 
Q 79*426  T 33654  ☒ 342320
Q 26*948  T 24648  ☒ 245222
Q 559*19  T 10621  ☒ 10077 
Q 231*45  T 10395  ☒ 100550

--------------------------------------------------
Iteration 27
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 216us/step - loss: 1.1182 - acc: 0.4743 - val_loss: 1.1500 - val_acc: 0.4597
Q 735*72  T 52920  ☒ 519300
Q 397*769 T 305293 ☒ 311681
Q 219*112 T 24528  ☒ 22178 
Q 51*93   T 4743   ☒ 4781  
Q 55*203  T 11165  ☒ 100550
Q 0*393   T 0      ☑ 0     
Q 640*781 T 499840 ☒ 492260
Q 316*453 T 143148 ☒ 137872
Q 10*819  T 8190   ☒ 83700 
Q 813*94  T 76422  ☒ 78668 

--------------------------------------------------
Iteration 28
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 223us/step - lo

18000/18000 [==============================] - 4s 217us/step - loss: 1.0333 - acc: 0.5070 - val_loss: 1.0828 - val_acc: 0.4649
Q 475*543 T 257925 ☒ 259225
Q 665*189 T 125685 ☒ 125335
Q 894*421 T 376374 ☒ 361554
Q 961*75  T 72075  ☒ 702750
Q 58*159  T 9222   ☒ 85128 
Q 747*64  T 47808  ☒ 470244
Q 7*596   T 4172   ☒ 4194  
Q 201*378 T 75978  ☒ 795888
Q 672*675 T 453600 ☒ 441550
Q 872*54  T 47088  ☒ 460244

--------------------------------------------------
Iteration 41
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 202us/step - loss: 1.0236 - acc: 0.5096 - val_loss: 1.0764 - val_acc: 0.4762
Q 864*2   T 1728   ☒ 1782  
Q 35*710  T 24850  ☒ 255500
Q 32*134  T 4288   ☒ 45564 
Q 93*399  T 37107  ☒ 367110
Q 719*717 T 515523 ☒ 517333
Q 90*636  T 57240  ☒ 568400
Q 14*182  T 2548   ☒ 25584 
Q 231*45  T 10395  ☒ 100550
Q 256*81  T 20736  ☒ 200880
Q 362*108 T 39096  ☒ 435564

--------------------------------------------------
Iteration 

Q 448*18  T 8064   ☒ 8224  
Q 218*937 T 204266 ☒ 208016
Q 358*77  T 27566  ☒ 270120
Q 352*82  T 28864  ☒ 280028
Q 632*87  T 54984  ☒ 557024
Q 835*21  T 17535  ☒ 178450
Q 47*471  T 22137  ☒ 224010
Q 72*119  T 8568   ☒ 84728 
Q 220*52  T 11440  ☒ 118400
Q 237*742 T 175854 ☒ 171002

--------------------------------------------------
Iteration 54
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 203us/step - loss: 0.9644 - acc: 0.5365 - val_loss: 1.0460 - val_acc: 0.4979
Q 900*93  T 83700  ☒ 81100 
Q 84*900  T 75600  ☒ 752000
Q 366*496 T 181536 ☒ 171352
Q 801*951 T 761751 ☒ 730039
Q 6*354   T 2124   ☒ 21468 
Q 507*201 T 101907 ☒ 105537
Q 26*948  T 24648  ☒ 245480
Q 943*9   T 8487   ☒ 8457  
Q 251*85  T 21335  ☒ 204450
Q 103*17  T 1751   ☒ 2227  

--------------------------------------------------
Iteration 55
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 201us/step - lo

18000/18000 [==============================] - 4s 198us/step - loss: 0.8282 - acc: 0.6061 - val_loss: 0.9307 - val_acc: 0.5596
Q 15*681  T 10215  ☒ 100450
Q 106*62  T 6572   ☒ 6012  
Q 894*400 T 357600 ☒ 359600
Q 609*224 T 136416 ☒ 131406
Q 25*122  T 3050   ☒ 26500 
Q 34*113  T 3842   ☒ 3272  
Q 586*62  T 36332  ☒ 36152 
Q 88*402  T 35376  ☒ 348560
Q 535*930 T 497550 ☒ 494850
Q 150*589 T 88350  ☒ 801500

--------------------------------------------------
Iteration 68
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 196us/step - loss: 0.8075 - acc: 0.6180 - val_loss: 0.8955 - val_acc: 0.5768
Q 9*889   T 8001   ☒ 8091  
Q 23*441  T 10143  ☒ 10243 
Q 825*87  T 71775  ☒ 706750
Q 48*56   T 2688   ☒ 27480 
Q 842*7   T 5894   ☒ 59646 
Q 231*491 T 113421 ☒ 118471
Q 446*317 T 141382 ☒ 143042
Q 463*17  T 7871   ☒ 7391  
Q 888*399 T 354312 ☒ 348552
Q 967*537 T 519279 ☒ 513329

--------------------------------------------------
Iteration 

Q 32*948  T 30336  ☒ 304360
Q 707*445 T 314615 ☒ 311335
Q 78*782  T 60996  ☒ 612160
Q 688*387 T 266256 ☒ 262456
Q 446*317 T 141382 ☒ 143002
Q 10*704  T 7040   ☒ 70400 
Q 264*901 T 237864 ☒ 232664
Q 892*770 T 686840 ☒ 690140
Q 2*382   T 764    ☒ 784   
Q 701*99  T 69399  ☒ 68879 

--------------------------------------------------
Iteration 81
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 199us/step - loss: 0.7393 - acc: 0.6275 - val_loss: 0.8669 - val_acc: 0.5788
Q 21*440  T 9240   ☒ 82400 
Q 893*922 T 823346 ☒ 832346
Q 159*848 T 134832 ☒ 131132
Q 377*18  T 6786   ☒ 6006  
Q 377*763 T 287651 ☒ 286301
Q 936*482 T 451152 ☒ 440252
Q 154*862 T 132748 ☒ 131008
Q 456*3   T 1368   ☒ 1328  
Q 1*53    T 53     ☑ 53    
Q 884*961 T 849524 ☒ 830624

--------------------------------------------------
Iteration 82
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 193us/step - lo

18000/18000 [==============================] - 4s 200us/step - loss: 0.6845 - acc: 0.6443 - val_loss: 0.8712 - val_acc: 0.5655
Q 562*930 T 522660 ☒ 511460
Q 701*99  T 69399  ☒ 68879 
Q 4*837   T 3348   ☒ 338844
Q 691*749 T 517559 ☒ 510539
Q 211*72  T 15192  ☒ 147526
Q 36*259  T 9324   ☒ 94448 
Q 25*462  T 11550  ☒ 113500
Q 316*10  T 3160   ☒ 3660  
Q 564*2   T 1128   ☒ 10884 
Q 88*402  T 35376  ☒ 353960

--------------------------------------------------
Iteration 95
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 207us/step - loss: 0.6794 - acc: 0.6481 - val_loss: 0.8500 - val_acc: 0.5826
Q 772*81  T 62532  ☒ 630520
Q 14*550  T 7700   ☑ 7700  
Q 22*212  T 4664   ☒ 45446 
Q 32*688  T 22016  ☒ 220960
Q 149*347 T 51703  ☒ 52233 
Q 407*61  T 24827  ☒ 244271
Q 159*848 T 134832 ☒ 135132
Q 55*203  T 11165  ☒ 110150
Q 650*1   T 650    ☒ 750   
Q 434*60  T 26040  ☒ 250400

--------------------------------------------------
Iteration 

# Testing

In [18]:
def vectorization(ary_x, ary_y):
    print('Vectorization...')
    x = np.zeros((len(test_x), MAXLEN, len(chars)), dtype=np.bool)
    y = np.zeros((len(test_y), DIGITS + 1, len(chars)), dtype=np.bool)
    for i, sentence in enumerate(test_x):
        x[i] = ctable.encode(sentence, MAXLEN)
    for i, sentence in enumerate(test_y):
        y[i] = ctable.encode(sentence, DIGITS + 1)
    return x, y

In [19]:
def evaluation_test(x_test, y_test, model):
    x_val, y_val = vectorization(test_x, test_y)
    for i in range(len(y_val)):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

In [20]:
print("MSG : Prediction")
#####################################################
## Try to test and evaluate your model ##############
# test_x = ["555+175", "860+7  ", "340+29 "]
# test_y = ["730 ", "867 ", "369 "] 

# evaluation_test(test_x, test_y, model)
#####################################################
testing_callback = model.evaluate(x=test_x, y=test_y, batch_size=1, verbose=1)
for i in range(len(model.metrics_names)):
        print(model.metrics_names[i]+" : "+str(testing_callback[i]))

MSG : Prediction
60000/60000 [==============================] - 124s 2ms/step
loss : 0.8532441508709965
acc : 0.5779555603307982
